<a href="https://colab.research.google.com/github/sugiyama404/ReinforcementLearningForGymOrAtari/blob/main/Control/multi_car_racing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/igilitschenski/multi_car_racing.git
%cd multi_car_racing
!pip uninstall -y imgaug & install imgaug=0.2.7
!pip install -e .
!python -m gym_multi_car_racing.multi_car_racing
!pip uninstall gym -y > /dev/null 2>&1 # gym 0.17.3 was broken 2021/11/08
!pip install gym > /dev/null 2>&1

Cloning into 'multi_car_racing'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 28 (delta 12), reused 14 (delta 3), pack-reused 0
Unpacking objects: 100% (28/28), done.
/content/multi_car_racing
install: missing destination file operand after 'imgaug=0.2.7'
Try 'install --help' for more information.
Found existing installation: imgaug 0.2.9
Uninstalling imgaug-0.2.9:
  Successfully uninstalled imgaug-0.2.9
Obtaining file:///content/multi_car_racing
     |████████████████████████████████| 448 kB 5.0 MB/s 
     |████████████████████████████████| 1.0 MB 60.0 MB/s 
  Attempting uninstall: shapely
    Found existing installation: Shapely 1.8.0
    Uninstalling Shapely-1.8.0:
      Successfully uninstalled Shapely-1.8.0
  Running setup.py develop for gym-multi-car-racing
/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'gym_multi_car_racing.multi_car_racing' found in sys.modules after impor

In [2]:
!apt update  > /dev/null 2>&1
!apt install xvfb  > /dev/null 2>&1
!pip install pyvirtualdisplay  > /dev/null 2>&1
from pyvirtualdisplay import Display
d = Display()
d.start()

import gym
from gym import wrappers
import gym_multi_car_racing
import numpy as np

In [3]:
class FlatMultiDisplayWrapper(gym.Wrapper):
    def __init__(self, env, nrows=1, ncols=1, empty_cell=0):
        super().__init__(env)
        self.nrows = nrows
        self.ncols = ncols
        self.empty_cell = empty_cell

    def render(self, mode="human", **kwargs):
        stack_images = super().render(mode, **kwargs)
        nstack = stack_images.shape[0]
        frame_shape = stack_images.shape[1:]

        if len(frame_shape) >= 2:
            self.flat_shape = list(frame_shape)
            self.flat_shape[0] = self.flat_shape[0]*self.nrows
            self.flat_shape[1] = self.flat_shape[1]*self.ncols
            frame = np.full(self.flat_shape, self.empty_cell, dtype=np.uint8)

            for i in range(min(nstack, self.nrows*self.ncols)):
                row = i // self.ncols
                col = i % self.ncols
                frame[row*frame_shape[0]:(row+1)*frame_shape[0],
                      col*frame_shape[1]:(col+1)*frame_shape[1]] = stack_images[i]
        else:
            # たまに、空の画像を stack した画像が返ってくる。
            frame = np.full(self.flat_shape, self.empty_cell, dtype=np.uint8)

        return frame

    def step(self, action):
        obs, rew, done, info = super().step(action)
        # Reward が gym.Env の仕様を満たしていないので変換する。元の値はinfoに入れておく
        info["original_reward"] = rew
        return obs, rew.sum(), done, info

In [5]:
actions = np.array([[ 0, 0, 0],  # [0]: straight
                    [ 0, 1, 0],  # [1]: acceleration
                    [ 0, 0, 1],  # [2]: decelerate
                    [ 1, 0, 0],  # [3]: Turn right
                    [-1, 0, 0]]) # [4]: Turn left

env = gym.make("MultiCarRacing-v0", num_agents=2)
env = FlatMultiDisplayWrapper(env)
env = wrappers.Monitor(env, './', force = True)
obs = env.reset()
done = False
total_reward = 0

while not done:
    action = np.random.randint(5)
    action = actions[action]
    obs, reward, done, info = env.step([action,action])
    total_reward += reward
    env.render()

print("individual scores:", total_reward)

/usr/local/lib/python3.7/dist-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
  "Box bound precision lowered by casting to {}".format(self.dtype)


Track generation: 1135..1423 -> 288-tiles track
individual scores: -149.47735191637523
